# Segmenting and Clustering Neighborhoods in Toronto_Assignment 1_Part1

### The following code in this Notebook scrapes data from a source and transfroms the data into dataframe 

In [27]:
import requests
import csv
from bs4 import BeautifulSoup    # importing Required modules/libraries

#### _Extracting the HTML Content From the the URL and accessing the table content data_

In [28]:
# Code to extract Columns and rows of the table content data

source_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # sourcepath
r = requests.get(source_path) #get the HTML contect

# creates a Parse tree from Raw HTML Content
soup = BeautifulSoup(r.content, 'lxml') # Pass the raw HTML contect and HTML Parser(in this case we are using 'lxml') to BeautifulSoup Object

#print(soup.prettify())

table = soup.find('table',{'class':'wikitable sortable'}) # finding the table content from the soup Oject

#Extracting table headers(enclosed in th) and rows(enclosed with in<td> the data
headers1 = [header.text for header in table.find_all('th')] 
#print(headers1)

rows_data = [] 

for row in table.find_all('tr'):
   rows_data.append([val.text.encode('utf8') for val in row.find_all('td')]) #val.text.encode('utf8')

#The data is written to CSV File 
with open('toronto_postal_info.csv', 'w') as f:
     writer = csv.writer(f)
     writer.writerow(headers1)
     writer.writerows(row for row in rows_data if row)
     print("the csv file is written")



the csv file is written


#### _Get the data from the CSV load to pandas DataFrame_

In [29]:
import os
print(os.getcwd())

# Display all of the files found in your current working directory
print(os.listdir(os.getcwd()))

/home/dsxuser/work
['toronto_postal_info.csv']


In [30]:
import pandas as pd

df_data=pd.read_csv("toronto_postal_info.csv")


df_data.head(5)

,Postcode,Borough,Neighbourhood
0,b'M1A',b'Not assigned',b'Not assigned\n'
1,b'M2A',b'Not assigned',b'Not assigned\n'
2,b'M3A',b'North York',b'Parkwoods\n'
3,b'M4A',b'North York',b'Victoria Village\n'
4,b'M5A',b'Downtown Toronto',b'Harbourfront\n'


#### _Cleaning the Data_
1. Removing unwanted Characters
2. Ignore cells with a borough that is Not assigned.(drop the rows with borough not assigned)
3. If a row has a borough but a Not assigned neighborhood, then the neighborhood's value will be the same as the borough. 


In [33]:
# 1
#Stripping characters('',b,\n) from the DataFrame
df_data = df_data.apply(lambda x: x.str.strip(r"'b\\n\"")) # removing \n,'b and double quotes from the data
df_data.columns = df_data.columns.str.strip(r"'b\\n\"")
df_data.columns=df_data.columns.str.lstrip(r"'b\\n\"")
df_data.columns = df_data.columns.str.strip("\n")# removes\n from column name's # can also use rename f['col$'].name = 'col'
#print (df_data)

df_data.head(5)
#df_data.shape #(289,3)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [38]:
# 2

# finding the data where colums Borough and Neighbourhood ='Not assigned' and drop them form the data set

# these rows will not be processed

indexNames = df_data[ (df_data['Neighbourhood'].str.contains('Not assigned') ) & (df_data['Borough'] .str.contains('Not assigned')) ].index
df_data.drop(indexNames , inplace=True)

df_data.shape

(212, 3)

In [36]:
# 3 
# If a row has a borough but a Not assigned neighborhood, then the neighborhood's value will be the same as the borough. 
# Checking for data where Neighbourhood = 'Not assigned'
df_data[(df_data['Neighbourhood'].str.contains('Not assigned'))]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [40]:
# Copy Borough Value to Neighborhood where Neighborhood='Not assigned'

df_data.loc[df_data['Neighbourhood']=='Not assigned', ['Neighbourhood']] =df_data.loc[8,"Borough"]

df_data.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malver


#### _Grouping  the data and combinig Neighbourhoods seperated with , that fall under same Postcode_

In [42]:
group_postcode=df_data.groupby(['Postcode','Borough','Neighbourhood']).count()
group_postcode.head(10)

#g1 = df1.groupby( [ "Name", "City"] ).count()

Empty DataFrame
Columns: []
Index: [(M1B, Scarborough, Malver), (M1B, Scarborough, Rouge), (M1C, Scarborough, Highland Creek), (M1C, Scarborough, Port Unio), (M1C, Scarborough, Rouge Hill), (M1E, Scarborough, Guildwood), (M1E, Scarborough, Morningside), (M1E, Scarborough, West Hill), (M1G, Scarborough, Wobur), (M1H, Scarborough, Cedarbrae)]

In [46]:
# combining the Neighbourd's that falls under same postal code

df_postaldata=df_data.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : ','.join(x))

df_postaldata.head(20)

#df_postaldata['Postcode'].unique()
#print(df_postaldata.set_index(['Postcode']).index.is_unique)--checks if the col is unique -boolean value
#df_postaldata['Postcode'].unique()-- unique value

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malver"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Unio"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Wobur
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [50]:
# Saving the data to CSV
df_postaldata.to_csv('toronto_postaldata.csv')
print("Saving to csv")

Saving to csv


### **Print the shape of the cleansed final dataframe**

In [51]:
df_postaldata.shape

(103, 3)